In [1]:
from idaes import dmf
from idaes.dmf.resource import TidyUnitData
from idaes.core.util.misc import extract_data

In [2]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
from idaes.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)

In [3]:
import hda_ideal_VLE_dmf as thermo_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

In [5]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()

In [7]:
from idaes.dmf import magics

In [10]:
%dmf init .

*Success!* Using workspace at "."

In [11]:
%dmf list

| ID | Name(s) | Type | Modified | Description | 
| -- | ------- | ---- | -------- | ----------- |
| 623804ff69564eda9f92ae3871ed8687 | pressure_crit | propertydb | 3 days ago | Critical pressure |
| b0b333fdd4574e4eae1c4cef98c19ee8 | temperature_crit | propertydb | 11 hours ago | Critical temperature |
| d9b59dffe0e4428baab2f80f42c62c3c | mw | propertydb | 9 hours ago | Molecular weight |
| 30ad5a72c576471b8b0a6efcab1114c8 | cp_liq | propertydb | 9 hours ago | Liquid densities |
| 4ef9d2ce193f4a0487763fbe4b94c347 | bp | propertydb | 7 hours ago | Pure component boiling points at standard pressure [K] |
| 5bc493a1ccb74639b0403d0d66924b51 | cp_ig | propertydb | 7 hours ago | Specific heat capacity, enthalpy |
| 6ff47009063047c58d70779374175d9c | pressure_sat_coeff | propertydb | 6 hours ago | Pressure saturation coefficient |
| 15907525b89643c08bf9bc0a0b8e2fe6 | dh_vap | propertydb | 2 minutes ago | Heat of vaporization |

True

In [6]:
wspace = dmf.DMF(".")

In [8]:
dh_vap = wspace.fetch_one(rid='15907525b89643c08bf9bc0a0b8e2fe6')

In [9]:
dh_vap_data = dmf.resource.TidyUnitData(dh_vap.data)

In [10]:
dh_vap_data.table

,compound,value
0,benzene,33870.0
1,toluene,38262.0
2,hydrogen,0.0
3,methane,0.0
4,diphenyl,62710.0


In [11]:
dh_vap_data.units

[None, 'K']

In [12]:
cp_ig = wspace.find_one(name="cp_ig")

In [13]:
cp_ig_data = dmf.resource.TidyUnitData(cp_ig.data)

In [14]:
cp_ig_data.table

,phase,compound,n,value
0,Liq,benzene,1,1.290000e+05
1,Liq,benzene,2,-1.700000e+02
2,Liq,benzene,3,6.480000e-01
3,Liq,benzene,4,0.000000e+00
4,Liq,benzene,5,0.000000e+00
5,Vap,benzene,1,-3.392000e+01
6,Vap,benzene,2,4.739000e-01
7,Vap,benzene,3,-3.017000e-04
8,Vap,benzene,4,7.130000e-08
9,Vap,benzene,5,0.000000e+00


In [16]:
cp_ig.v['sources']

[{'date': '2019',
  'isbn': '9780071834087',
  'language': 'en',
  'source': "Green, W. and Southard, M., ed. 'Perry's Chemical Engineer's Handbook', 9th Edition"}]